# Otimização da distribuição temporal de empregados em regime híbrido

O objetivo desse projeto é otimizar, através da utilização de algoritmos genéticos, a distribuição dos dias de trabalho presencial dos colaboradores de uma mesma empresa buscando:

1. Atingir o nível de ocupação do espaço desejado por dia da semana;
2. Garantir a alocação de gerências relacionadas nos mesmos dias de trabalho.

A lógica construída permite acomodar as seguintes restrições:

1. A capacidade física do espaço;
2. Ocupação desejada diferente por dia da semana;
3. Quantidade de dias presenciais exigidos diferente por gerência;
4. Fixação dos dias de trabalho presencial de determinadas gerências;
5. Seleção de combinações de dias não-permitidas;
6. Definição por pesos de qual critério de avaliação do objetivo é mais relevante (ocupação ou relacionamento entre gerências).

In [340]:
! pip install deap


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [341]:
import pandas as pd
import numpy as np
import random

from itertools import permutations

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

### Leitura dos arquivos .csv de input

O projeto depende de dois arquivos .csv de input

- `pessoas_por_gerencia.csv`: esse arquivo deve possuir os IDs de cada gerência, a sua quantidade de pessoas e a quantidade de dias obrigatórios.
- `relacionamentos.csv`: esse arquivo deve possuir, para cada gerência, a lista de IDs das gerências relacionadas, ou seja, aquelas que preferencialmente devem estar nos mesmos dias de presencial.

In [ ]:
# Lendo arquivos de input para obter quantidade de pessoas por gerência
df_pessoas_por_gerencia = pd.read_csv(
    'pessoas_por_gerencia.csv',
    sep=';',
    dtype={'ID': int, 'ID GG': int, 'Gerência': str, 'Classificação': str, 'Quantidade': int, 'Dias obrigatórios': int}
).fillna(0)

df_relacionamentos = pd.read_csv('relacionamentos.csv', sep=";")

### Constantes

In [343]:
QTD_DIAS_SEMANA = 5
COLUNAS_DIAS = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']
COLUNA_QTD = 'Quantidade' # Coluna do arquivo pessoas_por_gerencia com a quantidade

# Definindo constantes a partir dos arquivos lidos

QUANTIDADE_EMPREGADOS = df_pessoas_por_gerencia[COLUNA_QTD].sum()
COUNT_GERENCIAS = df_pessoas_por_gerencia['ID'].count()
MIN_ID_GERENCIA, MAX_ID_GERENCIA = df_pessoas_por_gerencia['ID'].min(), df_pessoas_por_gerencia['ID'].max()
DIAS_PRESENCIAIS_POR_GERENCIA = df_pessoas_por_gerencia['Dias obrigatórios'].to_list()

RELACIONAMENTOS = df_relacionamentos.set_index('ID').to_dict()['Relacionamentos']
RELACIONAMENTOS = {k: [int(n) for n in RELACIONAMENTOS[k].split(',')] for k in RELACIONAMENTOS.keys()}

### Inputs do usuário

In [ ]:
# Capacidade máxima de pessoas
CAPACIDADE_PREDIO = 290

# Lista de inteiros que representam com a quantidade de dias da semana possíveis
# para distribuição.
QTD_DIAS_PRESENCIAIS_POSSIVEIS = [2, 3]

# Lista de combinações de dias proibidas. Cada combinação é uma lista de 5 booleanos
# representados por 0 ou 1, indicando se o dia da semana correspondente àquela posição 
# é permitido (1) ou não (0).
COMBINACOES_DIAS_PROIBIDAS = [
    # Combinações com dois dias de presencial
    [0,1,1,0,0], # obrigatório ter pelo menos segunda ou sexta
    [0,0,1,1,0], # obrigatório ter pelo menos segunda ou sexta
    [0,1,0,1,0], # obrigatório ter pelo menos segunda ou sexta
    [1,0,0,0,1], # proibir segunda e sexta
    [1,0,1,0,0], # sem terceirizados na quarta
    [0,0,1,0,1], # sem terceirizados na quarta
    # Combinações com três dias de presencial
    [0,1,1,1,0], # obrigatório ter pelo menos segunda ou sexta
    [1,0,0,1,1], # proibir segunda e sexta
    [1,1,0,0,1], # proibir segunda e sexta
    [1,0,1,0,1], # proibir segunda e sexta
]

# Ocupação do andar desejada por dia (representada por um valor percentual de 0 a 1
# aplicado ao dia da semana correspondente àquela posição)
OCUPACAO_DESEJADA_POR_DIA = np.array([0.9, 0.9, 1.0, 0.9, 0.9])

# Gerências que têm dias fixos. Cada chave é o índice da gerência e o valor é uma
# lista de 5 booleanos representados por 0 ou 1, indicando se o dia da semana 
# correspondente àquela posição é permitido (1) ou não (0).
GERENCIAS_FIXAS = {
    8: [1,1,1,0,0],
    12: [1,1,1,0,0],
}

# Pesos para nivelamento dos critérios de otimização usados na função objetivo
PESO_OCUPACAO = 1 # Peso do critério ocupação do andar
PESO_RELACIONAMENTOS = 1 # Peso do critério gerências relacionadas no mesmo dia

In [345]:
# Construindo combinações de dias dinamicamente para todas as quantidades obrigatórias de dias possíveis

QTD_DIAS_PRESENCIAIS_POSSIVEIS.sort()

combinacoes_dias = {}

for qtd_dias_presenciais in QTD_DIAS_PRESENCIAIS_POSSIVEIS:
    lista_dias = [1] * (qtd_dias_presenciais) + [0] * (QTD_DIAS_SEMANA - qtd_dias_presenciais)
    conjunto = set(permutations(lista_dias)) # uso do conjunto pra eliminar duplicatas
    combinacoes_dias[qtd_dias_presenciais] = [list(t) for _,t in enumerate(conjunto) if not list(t) in COMBINACOES_DIAS_PROIBIDAS]

print(combinacoes_dias)

{2: [[1, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0]], 3: [[0, 1, 0, 1, 1], [1, 1, 1, 0, 0], [0, 0, 1, 1, 1], [1, 0, 1, 1, 0], [0, 1, 1, 0, 1], [1, 1, 0, 1, 0]]}


### Geração de indivíduos

O indivíduo para esse problema é representado por uma matriz com N linhas e 5 colunas, sendo N a quantidade de gerências indicadas no arquivo `pessoas_por_gerencia.csv`. 

Cada linha da matriz representa uma gerência e cada coluna, um dia da semana. O valor de cada célula da matriz corresponde, portanto, à quantidade de pessoas da gerência trabalhando presencialmente naquele dia da semana.

Visando simplificar o problema, ao invés de realizar a seleção aleatória de cada um dos Nx5 valores durante a criação do indivíduo, decidiu-se selecionar aleatoriamente ***a combinação de dias de cada gerência*** e preencher a matriz a partir daí, multiplicando os valores da linha de cada gerência pela sua respectiva quantidade de pessoas.

Essa abordagem reduz significativamente a quantidade de indivíduos inválidos possíveis de serem gerados, diminuindo também a quantidade de restrições que precisam ser impostas no modelo e acelera a convergência. Antes de realizar a implementação em Python e `deap` foram feitas tentativas com o Solver que esbarraram em dificuldades oriundas justamente dessas condições.

In [346]:
# Função auxiliar para forçar a combinação das gerências com dias fixos 
def substituir_gerencias_fixas(individuo):
    new_ind = individuo
    for (id_gerencia, dias) in GERENCIAS_FIXAS.items():
        new_ind[id_gerencia] = (np.array(dias) * df_pessoas_por_gerencia[COLUNA_QTD][id_gerencia]).tolist()
    return new_ind

# Função geradora de indivíduos do algoritmo genético
def gerador_individuo(icls):
    combinacoes_por_gerencia = []
    for i in range(0, COUNT_GERENCIAS):
        qtd_dias_presenciais = DIAS_PRESENCIAIS_POR_GERENCIA[i]
        lista_combinacoes_dias_possiveis = combinacoes_dias[qtd_dias_presenciais]
        combinacoes_por_gerencia.append(random.choice(lista_combinacoes_dias_possiveis))

    combinacoes_por_gerencia = np.array(combinacoes_por_gerencia)
    pessoas_por_dia_da_semana_gerencia = combinacoes_por_gerencia * df_pessoas_por_gerencia[COLUNA_QTD].to_numpy()[:, None]
    
    individuo = substituir_gerencias_fixas(pessoas_por_dia_da_semana_gerencia.tolist())
    return icls(individuo)

### Função de restrição

Em função da simplificação empregada na geração do indivíduo, a única restrição que precisou ser implementada foi a obrigação de respeitar a ocupação máxima desejada de cada andar por dia, considerando os fatores inputados pelo usuário na variável `OCUPACAO_DESEJADA_POR_DIA`.

Retorna `true` se o individuo atender à função de restrição, `false` caso contrário.

In [ ]:
def funcao_de_restricao(individuo):
    np_array = np.array(individuo)
    ocupacao_por_dia = np_array.sum(axis=0)
    andar_nao_lotado = ocupacao_por_dia <= OCUPACAO_DESEJADA_POR_DIA*CAPACIDADE_PREDIO
    return andar_nao_lotado.all()

### Função objetivo

A função objetivo avalia dois critérios de otimização:

1. Manter o prédio com ocupação o mais constante possível ao longo da semana, considerando a possível variação de ocupação desejada por dia informada em `OCUPACAO_DESEJADA_POR_DIA`
2. Atendimento aos relacionamentos informados no arquivo `relacionamentos.csv`

Foram feitas tentativas de otimização multi-objetivo com o DEAP, utilizando NSGA-II, conforme exemplos [1], [2] e [3]. Contudo, a biblioteca não conseguia lidar bem com as funções de restrição nesse cenário e diversos indivíduos que deveriam ser considerados inválidos (ocupação superava a capacidade do prédio) estavam compondo o pareto (ParetoFront) indicativo das melhores soluções.

- [1] https://deap.readthedocs.io/en/master/examples/ga_knapsack.html
- [2] https://advancedoracademy.medium.com/multi-objective-genetic-algorithms-moga-with-deap-ba1cb3578887
- [3] https://medium.com/@rossleecooloh/optimization-algorithm-nsga-ii-and-python-package-deap-fca0be6b2ffc

Uma investigação na internet retornou que houve em algum momento uma dificuldade realmente no desenvolvimento da biblioteca relacionado a esse ponto ([4]), mas não há relato sobre correções desse problema.

- [4] https://github.com/deap/deap/issues/30

Considerando essa restrição, a função objetivo foi modelada como a soma de dois fatores que variam entre zero e um: `fitness_ocupacao` e `fitness_relacionamentos`. Caso o usuário tenha interesse em dar um peso maior para algum desses fatores, ele pode fazê-lo alterando as variáveis `PESO_OCUPACAO` e `PESO_RELACIONAMENTOS`.

O fator `fitness_ocupacao` foi calculado usando a fórmula f(v)=exp(−v^k/s^k), em que `v` é a variância e `s` e `k` são constantes para ajustar quão rápido o valor converge para zero. Essa fórmula foi uma sugestão obtida em [5] como uma alternativa de medida de dispersão sempre com valor entre 0 e 1, a fim de mantê-la compatível com o outro objetivo.

- [5] https://math.stackexchange.com/questions/2833062/a-measure-similar-to-variance-thats-always-between-0-and-1

O fator `fitness_relacionamentos` foi calculado dividindo a "quantidade de relacionamentos respeitados" pela "quantidade total de relacionamentos informados". Para que um relacionamento seja respeitado, todos os colaboradores das duas gerências precisam estar presentes nos mesmos dias. Caso as duas gerências tenham quantidades diferentes de dias presenciais exigidos, vale o menor para essa avaliação.

In [ ]:
def funcao_objetivo(individuo):
    # Fitness por ocupação do dia da semana
    np_array = np.array(individuo)
    ocupacao_por_dia = np_array.sum(axis=0)/(OCUPACAO_DESEJADA_POR_DIA*CAPACIDADE_PREDIO)
    fitness_ocupacao = np.exp(-np.var(ocupacao_por_dia)**2 / 0.01**2) # fórmula usada para medir a dispersão da ocupação por dia entre 0 e 1
    
    # fitness pelos relacionamentos
    qtd_relacionamentos_respeitados = 0
    # Para analisar se gerências relacionadas com quantidades diferentes de dias presenciais obrigatórios estão
    # presentes nos mesmos dias, o algoritmo começa pelas gerências com menos dias presenciais e soma a quantidade de
    # de pessoas de todas as gerências relacionadas em cada dia. Se houver uma quantidade de dias com a soma de pessoas
    # de todas as gerências igual à quantidade de dias presenciais exigido, então o relacionamento está sendo respeitado.
    # Na sequência, o algoritmo parte pra próxima quantidade de dias e descarta as gerências que já foram analisadas,
    # repetindo a lógica. 
    for qtd_dias_presenciais in QTD_DIAS_PRESENCIAIS_POSSIVEIS: # (lista de dias presenciais possíveis precisa estar ordenada)
        # Inicia filtrando apenas os relacionamentos das gerências que têm a quantidade de dias presenciais especificada
        gerencias_analisaveis = df_pessoas_por_gerencia[df_pessoas_por_gerencia['Dias obrigatórios'] == qtd_dias_presenciais]['ID']
        gerencias_descartadas = df_pessoas_por_gerencia[df_pessoas_por_gerencia['Dias obrigatórios'] < qtd_dias_presenciais]['ID']
        relacionamentos_analisaveis = {gerencia: RELACIONAMENTOS[gerencia] for gerencia in gerencias_analisaveis}
        for (gerencia, gerencias_relacionadas) in relacionamentos_analisaveis.items():
            indices = gerencias_relacionadas + [gerencia]
            # Excluindo gerências que não possuem dias obrigatórios suficientes
            indices = list(set(indices).difference(set(gerencias_descartadas)))
            # Se após a exclusão resta apenas o relacionamento consigo mesmo, não há nada a ser feito
            if len(indices) <= 1:
                qtd_relacionamentos_respeitados = qtd_relacionamentos_respeitados + relacionamento_respeitado
                continue
            # Soma a quantidade de pessoas por dia de todas as gerências relacionadas
            pessoas_relacionadas_por_dia = np_array[indices, :].sum(axis=0)
            # Soma o total de pessoas de todas as gerências relacionadas
            max_pessoas_relacionadas = df_pessoas_por_gerencia.iloc[indices][COLUNA_QTD].sum()
            # Avalia em quantos dias a quantidade de pessoas das gerências é igual ao somatório total de pessoas dessas
            # gerências. Caso a quantidade de dias for igual a quantidade de dias presenciais obrigatórios, o relacionamento
            # foi satisfeito.
            relacionamento_respeitado = len(pessoas_relacionadas_por_dia[pessoas_relacionadas_por_dia == max_pessoas_relacionadas]) == qtd_dias_presenciais
            qtd_relacionamentos_respeitados = qtd_relacionamentos_respeitados + relacionamento_respeitado

    fitness_relacionamentos = qtd_relacionamentos_respeitados / len(RELACIONAMENTOS)

    return fitness_ocupacao*PESO_OCUPACAO + fitness_relacionamentos*PESO_RELACIONAMENTOS,

### Função de mutação

Em alguns casos, conforme probabilidade de mutação informado na configuração do algoritmo (mutpb), o indivíduo sofrerá uma mutação, com a variação de algum dos seus cromossomos. Nesse caso, uma ou mais gerências terão a sua combinação de dias selecionada alterada.

No algoritmo implementado abaixo, temos uma probabilidade específica de cada cromossomo ser alterado (indpb), caso em que uma nova combinação de dias é sorteada. Importante atentar que, em função da restrição imposta pelo próprio problema, as gerências com dias fixos não podem sofrer mutação.

In [349]:
def mutacao(individuo, indpb):
    individuo_valido = False
    while not individuo_valido:
        new_ind = individuo
        for nr_gerencia in range(0, len(individuo)):
            if random.random() <= indpb and nr_gerencia not in GERENCIAS_FIXAS.keys():
                qtd_dias_presenciais = DIAS_PRESENCIAIS_POR_GERENCIA[nr_gerencia]
                lista_combinacoes_dias_possiveis = combinacoes_dias[qtd_dias_presenciais]
                novos_dias = np.array(random.choice(lista_combinacoes_dias_possiveis))
                mutacao = novos_dias*df_pessoas_por_gerencia[COLUNA_QTD][nr_gerencia]
                new_ind[nr_gerencia] = mutacao.tolist()
        individuo_valido = funcao_de_restricao(new_ind)
    return new_ind,

### Função de crossover

Para a função de crossover, foi necessário fazer uma modificação no método padrão do `deap` pois como cada linha (cromossomo) tem a quantidade de empregados de cada gerência trabalhando presencialmente por dia (coluna), não é possível simplesmente trocar as linhas de posição. Isso porque, ao fazer qualquer inversão nesse modelo, estaríamos levando a quantidade de colaboradores de uma gerência para outra, alterando as condições do problema.

Nesse cenário, a solução proposta foi obter a combinação de dias original de cada indivíduo e realizar o crossover deste objeto, respeitando, também, a restrição das gerências com dias fixados pelo usuário.

In [350]:
def crossover_two_point(icls, ind1, ind2):
    combinacao_dias_ind1 = pd.DataFrame(ind1).div(df_pessoas_por_gerencia[COLUNA_QTD], axis=0).fillna(0).astype(int)
    combinacao_dias_ind2 = pd.DataFrame(ind2).div(df_pessoas_por_gerencia[COLUNA_QTD], axis=0).fillna(0).astype(int)

    for qtd_dias_presenciais in QTD_DIAS_PRESENCIAIS_POSSIVEIS:
        gerencias_ind1 = combinacao_dias_ind1[(combinacao_dias_ind1 != 0).sum(axis=1) == qtd_dias_presenciais].index
        gerencias_ind2 = combinacao_dias_ind2[(combinacao_dias_ind2 != 0).sum(axis=1) == qtd_dias_presenciais].index
        
        ind1_parcial = combinacao_dias_ind1.iloc[gerencias_ind1][:].to_numpy()
        ind2_parcial = combinacao_dias_ind2.iloc[gerencias_ind2][:].to_numpy()

        size = min(len(ind1_parcial), len(ind2_parcial))

        cxpoint1 = random.randint(1, size)
        cxpoint2 = random.randint(1, size - 1)
        if cxpoint2 >= cxpoint1:
            cxpoint2 += 1
        else:  # Swap the two cx points
            cxpoint1, cxpoint2 = cxpoint2, cxpoint1

        ind1_parcial[cxpoint1:cxpoint2], ind2_parcial[cxpoint1:cxpoint2] \
            = ind2_parcial[cxpoint1:cxpoint2], ind1_parcial[cxpoint1:cxpoint2]
        
        combinacao_dias_ind1.loc[gerencias_ind1] = ind1_parcial
        combinacao_dias_ind2.loc[gerencias_ind2] = ind2_parcial
        
    ind1 = combinacao_dias_ind1.multiply(df_pessoas_por_gerencia[COLUNA_QTD], axis=0).values.tolist()
    ind2 = combinacao_dias_ind2.multiply(df_pessoas_por_gerencia[COLUNA_QTD], axis=0).values.tolist()

    ind1 = substituir_gerencias_fixas(ind1)
    ind2 = substituir_gerencias_fixas(ind2)

    return icls(ind1), icls(ind2)

### Configuração do deap

In [351]:
creator.create("Fitness", base.Fitness, weights=(1.0, ))
creator.create("Individual", list, fitness=creator.Fitness)

toolbox = base.Toolbox()
# Definir o gerador de numeros aleatórios de numeros inteiros entre o intervalo de IDs de gerência
toolbox.register("rand_int_gerencia_function", random.randint, MIN_ID_GERENCIA, MAX_ID_GERENCIA)

# Inicialização do cromossomo (quantos genes o cromossomo deve possuir)
toolbox.register("individual", gerador_individuo, creator.Individual)

# Registro do individuo na população
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registro do nome da função objetivo
toolbox.register("evaluate", funcao_objetivo)

# Registro da função de penalidade caso o individuo não obedeça as restrições
toolbox.decorate("evaluate", tools.DeltaPenalty(funcao_de_restricao, 0))

# Registro de qual o tipo de cruzamento deve ser utilizado (cruzamento de 2 pontos)
toolbox.register("mate", crossover_two_point, creator.Individual)

# Registro de qual tipo de mutação deve ser utilizado (probabilidade de um individuo sofrer mutação)
toolbox.register("mutate", mutacao, indpb=0.25)

# Registro de qual o tipo do método de seleção que será utilizado
toolbox.register("select", tools.selTournament, tournsize=5)

C:\Users\U4DD\AppData\Roaming\Python\Python312\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\U4DD\AppData\Roaming\Python\Python312\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [352]:
pop = toolbox.population(n=100)                           # inicialização da pop
hof = tools.HallOfFame(10)                                 # melhor indivíduo
stats = tools.Statistics(lambda ind: ind.fitness.values)  # estatísticas
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [353]:
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.4, mutpb=0.6, ngen=50, stats=stats, halloffame=hof, verbose=True) #aumentei mut = 0.7

gen	nevals	avg      	std     	min	max    
0  	100   	0.0165396	0.119347	0  	1.03184
1  	76    	0.515843 	0.420181	0  	1.34767
2  	84    	0.777853 	0.357372	0  	1.34767
3  	80    	0.837302 	0.349579	0  	1.38814
4  	74    	0.911374 	0.324718	0  	1.39161
5  	80    	0.94605  	0.369571	0  	1.40815
6  	75    	0.999059 	0.351337	0  	1.40815
7  	77    	0.982033 	0.379471	0  	1.40815
8  	83    	1.00783  	0.39109 	0  	1.46756
9  	69    	1.08365  	0.362166	0  	1.47661
10 	67    	1.05678  	0.419716	0  	1.48156
11 	73    	1.09568  	0.357456	0  	1.48156
12 	76    	1.06047  	0.413681	0  	1.56977
13 	81    	0.990249 	0.425727	0  	1.56977
14 	75    	1.06766  	0.461927	0  	1.56977
15 	74    	0.986977 	0.460352	0  	1.56977
16 	81    	1.03797  	0.430133	0  	1.56977
17 	82    	1.01791  	0.474568	0  	1.58876
18 	77    	1.02103  	0.485985	0  	1.58876
19 	71    	1.08093  	0.449994	0  	1.58876
20 	81    	1.10398  	0.452051	0  	1.64185
21 	77    	1.06268  	0.449758	0.150781	1.64185
22 	79    	1.14203  	0.432481

In [ ]:
# Melhor solução
print("Melhores Indivíduo:")

qtd_pessoas_com_total = df_pessoas_por_gerencia.copy()
qtd_pessoas_com_total.loc['Total'] = 'Total'

for ind in hof:
    melhor = pd.DataFrame(ind, columns=COLUNAS_DIAS)
    melhor.loc['Total'] = pd.Series(melhor.sum())
    resultado_completo = qtd_pessoas_com_total.join(melhor)[['Gerência', 'Classificação'] + COLUNAS_DIAS]
    display(resultado_completo)
    print(funcao_de_restricao(ind))
    print(funcao_objetivo(ind))

Melhores Indivíduo:


,Gerência,Classificação,Segunda,Terça,Quarta,Quinta,Sexta
0,ACADUP,Próprio,4,4,4,0,0
1,ACADUP,Terceirizado,0,0,0,0,0
2,AGP,Próprio,0,17,0,17,17
3,AGP,Terceirizado,0,1,0,0,1
4,AGUP,Próprio,0,0,61,61,61
5,AGUP,Terceirizado,0,0,0,13,13
6,DPOCOS,Próprio,3,3,3,0,0
7,DPOCOS,Terceirizado,0,0,0,0,0
8,EP,Próprio,52,52,52,0,0
9,EP,Terceirizado,8,8,0,0,0


True
(np.float64(1.756445391703764),)


,Gerência,Classificação,Segunda,Terça,Quarta,Quinta,Sexta
0,ACADUP,Próprio,4,4,4,0,0
1,ACADUP,Terceirizado,0,0,0,0,0
2,AGP,Próprio,0,0,17,17,17
3,AGP,Terceirizado,0,1,0,0,1
4,AGUP,Próprio,0,0,61,61,61
5,AGUP,Terceirizado,0,0,0,13,13
6,DPOCOS,Próprio,3,3,3,0,0
7,DPOCOS,Terceirizado,0,0,0,0,0
8,EP,Próprio,52,52,52,0,0
9,EP,Terceirizado,8,8,0,0,0


True
(np.float64(1.7432986319703403),)


...

True
(np.float64(1.6922288408828525),)


,Gerência,Classificação,Segunda,Terça,Quarta,Quinta,Sexta
0,ACADUP,Próprio,4,4,4,0,0
1,ACADUP,Terceirizado,0,0,0,0,0
2,AGP,Próprio,0,17,17,0,17
3,AGP,Terceirizado,0,1,0,0,1
4,AGUP,Próprio,0,0,61,61,61
5,AGUP,Terceirizado,0,0,0,13,13
6,DPOCOS,Próprio,3,3,3,0,0
7,DPOCOS,Terceirizado,0,0,0,0,0
8,EP,Próprio,52,52,52,0,0
9,EP,Terceirizado,0,0,0,8,8


True
(np.float64(1.691871957870955),)


,Gerência,Classificação,Segunda,Terça,Quarta,Quinta,Sexta
0,ACADUP,Próprio,4,4,4,0,0
1,ACADUP,Terceirizado,0,0,0,0,0
2,AGP,Próprio,0,17,17,0,17
3,AGP,Terceirizado,0,1,0,0,1
4,AGUP,Próprio,0,0,61,61,61
5,AGUP,Terceirizado,0,0,0,13,13
6,DPOCOS,Próprio,3,3,3,0,0
7,DPOCOS,Terceirizado,0,0,0,0,0
8,EP,Próprio,52,52,52,0,0
9,EP,Terceirizado,8,0,0,8,0


True
(np.float64(1.6914779870606358),)


In [ ]:
#### DEBUG

combinacoes_por_gerencia = np.array([random.randint(MIN_ID_COMBINACAO_DIAS, MAX_ID_COMBINACAO_DIAS) for i in range(0, COUNT_GERENCIAS)]).tolist()
# Cada linha representa uma gerência que recebe uma combinação de 3 dias na semana.
# O iloc preenche o dataframe com 1 para cada dia da semana da combinação
# Depois é preciso reiniciar o índice para um inteiro incremental e dropar a coluna de índice criada
dias_da_semana_por_gerencia = df_combinacoes_dias.iloc[combinacoes_por_gerencia].reset_index().drop(columns=['index', 'ID'])
# Multiplica-se todas as colunas pela quantidade de pessoas lotadas na gerência
# Depois soma-se todas as colunas para obter a ocupação de cada dia
pessoas_por_dia_da_semana_gerencia = dias_da_semana_por_gerencia.apply(lambda x: x*df_pessoas_por_gerencia[COLUNA_QTD])
lista = pessoas_por_dia_da_semana_gerencia.values.tolist()
np_array = np.array(lista)
print(np_array.sum(axis=0))
print((OCUPACAO_DESEJADA_POR_DIA*CAPACIDADE_PREDIO))
ocupacao_por_dia = np_array.sum(axis=0) <= (OCUPACAO_DESEJADA_POR_DIA*CAPACIDADE_PREDIO)
print(ocupacao_por_dia)
# ocupacao_ponderada_por_dia = ocupacao_por_dia * OCUPACAO_DESEJADA_POR_DIA # deve variar entre 0 e 5 
# display(np_array.sum(axis=0))

# display(andar_nao_lotado)
# display(andar_nao_lotado.all())
# display(funcao_de_restricao(lista))
# display(mutacao(lista, None, 0.5))
# display(pessoas_por_dia_da_semana_gerencia)
# display(pessoas_por_dia_da_semana_gerencia.sum(axis=0)/290)
# display(funcao_de_restricao(pessoas_por_dia_da_semana_gerencia))
# display(funcao_objetivo(pessoas_por_dia_da_semana_gerencia))

NameError: name 'MIN_ID_COMBINACAO_DIAS' is not defined

In [ ]:
pessoas_por_gerencia_dia = hof[0]

MIN_ID_SALA = df_capacidade_sala['ID'].min()
MAX_ID_SALA = df_capacidade_sala['ID'].max()

# para cada dia da semana, alocar equipe de cada gerência presente no dia nas salas
for id_gerencia in range(0, COUNT_GERENCIAS):
    # alocar pessoas na sala
    pessoas_sem_alocacao = np.max(pessoas_por_gerencia_dia[id_gerencia])
    salas_livres = df_capacidade_sala.index.to_list()
    while pessoas_sem_alocacao > 0:
        id_sala = random.choice(salas_livres)
        pessoas_sem_alocacao =         
